# Sheet properties

In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

# Imports

In [ ]:
import sys
sys.path.append('/home/rcendre/classification')
from numpy import array, logspace
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.svm import SVC
from sklearn.metrics import f1_score
from toolbox.classification.common import Data, Folds, IO, Tools
from toolbox.classification.parameters import Dermatology, Settings
from toolbox.transforms.common import FlattenTransform
from toolbox.transforms.labels import OrderedEncoder
from toolbox.views.common import Views, ViewsTools
from toolbox.models.models import DecisionVotingClassifier, ScoreVotingClassifier
from toolbox.transforms.images import SpatialImageTransform
import warnings
warnings.filterwarnings('ignore') 

# Parameters

In [ ]:
# Advanced parameters
use_unknown = False 
validation = 4
settings = Settings.get_default_dermatology()

# Sliding window

In [ ]:
inputs_options = []
inputs_options.append((250,0))
inputs_options.append((500,0))

# Low level predictions

In [ ]:
for size, overlap in inputs_options:
    inputs = IO.load(f'Spatial_{size}_{overlap}.pickle')
    
    # Add scaling
    Tools.predict(inputs, {'datum': 'Spatial'}, low_predictor, Tools.PREDICTION, mask=inputs.Type != 'Full')
    Tools.predict_proba(inputs, {'datum': 'Spatial'}, low_predictor, Tools.PROBABILITY, mask=inputs.Type != 'Full')

    Data.build_bags(inputs, inputs.Type == 'Full', 'Reference', inputs.Type == 'Instance', 'Source', Tools.PREDICTION)
    Data.build_bags(inputs, inputs.Type == 'Full', 'Reference', inputs.Type == 'Instance', 'Source', Tools.PROBABILITY)

    # Filter elements
    inputs = inputs[inputs.Type == 'Full']
    
    # Low Level
    IO.save(inputs, f'LowSpatial_{size}_{overlap}.pickle')